In [1]:
!pip install implicit

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import pandas as pd
import numpy as np
import math
import random
import matplotlib.pyplot as plt
%matplotlib inline
# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [3]:
data = pd.read_csv('retail_train.csv')
data.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [4]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [5]:
weight = data_train.groupby('item_id')['sales_value'].sum().reset_index()

In [6]:
weights = (weight['sales_value']/sum(weight['sales_value']))
items = data_train['item_id']

In [7]:
tuples = list(zip(items, weights))
df = pd.DataFrame(tuples,
                  columns = ['item_id', 'weight'])

In [8]:
def weighted_random_recommendation(df, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    items=(df['item_id'])
    weights=(df['weight'])
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    # your_code

    recs = np.random.choice(items ,p=weights, size=n,replace=False)
    
    return recs.tolist()

In [9]:
sum(weights)

0.9999999999995323

In [10]:
df

,item_id,weight
0,1004906,2.969296e-06
1,1033142,1.403822e-07
2,1036325,2.254623e-07
3,1082185,2.183723e-07
4,8160430,2.807644e-07
...,...,...
86860,5591538,0.000000e+00
86861,827683,1.132983e-06
86862,862139,6.366828e-07
86863,866778,3.545005e-07


In [11]:
%%time
# your_code
weighted_random_recommendation(df,n=5)

CPU times: total: 15.6 ms
Wall time: 3.99 ms


[1101589, 1133018, 823570, 1107824, 1038998]

### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [32]:
result = pd.read_csv('predictions_basic.csv')
result.head(10)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,[ 821867 834484 856942 865456 889248 ...,"[5586238, 1015228, 866118, 2416733, 2603573]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 878996, 826249]","[981760, 1127831, 1098066, 826249, 878996]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[161354, 63027, 1027802, 12263694, 307395]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 995242, 1029743, 840361, 961554]","[981760, 1004906, 961554, 1096036, 1080414]","[981760, 1004906, 859075, 1096036, 961554]","[999999, 1082185, 1098066, 6534178, 1127831]"
2,6,[ 920308 926804 946489 1006718 1017061 ...,"[13416054, 936084, 7410040, 9527114, 377218]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1098066, 826249, 1106523, 923746, 1058997]","[1098066, 826249, 860776, 854852, 1068719]","[1098066, 826249, 860776, 1068719, 916122]","[999999, 1082185, 1029743, 6534178, 1127831]"
3,7,[ 840386 889774 898068 909714 929067 ...,"[5574336, 990072, 868548, 995880, 842226]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 1098066, 840361, 883404, 916122]","[981760, 1098066, 883404, 1004906, 859075]","[981760, 883404, 1098066, 859075, 916122]","[999999, 1082185, 1029743, 1127831, 995785]"
4,8,[ 835098 872137 910439 924610 992977 ...,"[1277401, 94446, 3133282, 1925252, 855699]","[6534178, 6533889, 1029743, 6534166, 1082185]","[904360, 13115903, 13189726, 13190294, 15596515]","[904360, 5588666, 1096036, 979707, 1013321]","[904360, 1096036, 5588666, 979707, 1013321]","[999999, 1082185, 1029743, 1098066, 6534178]"
5,9,[ 864335 990865 1029743 9297474 10457112 ...,"[966310, 1114417, 896976, 15717035, 1054783]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 883404, 1133018, 1004906, 961554]","[981760, 883404, 961554, 1044078, 844179]","[981760, 883404, 1044078, 961554, 923746]","[999999, 1082185, 1029743, 1098066, 1127831]"
6,13,[ 6534178 1104146 829197 840361 862070 ...,"[911088, 13133787, 10311511, 59994, 15596516]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 1098066, 883404, 904360, 995785]","[981760, 1098066, 1044078, 883404, 962568]","[981760, 1098066, 1044078, 883404, 995785]","[999999, 1082185, 1029743, 6534178, 1127831]"
7,14,[ 840601 867293 933067 951590 952408 ...,"[1038153, 1119575, 1018818, 6039624, 12487492]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1082185, 840361, 1004906, 1106523, 961554]","[1082185, 961554, 1004906, 844179, 840361]","[1082185, 961554, 840361, 844179, 1004906]","[999999, 1029743, 1098066, 6534178, 1127831]"
8,15,[ 910439 1082185 959076 1023958 1082310 ...,"[1014751, 10456468, 10254529, 1219801, 9829777]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1098066, 840361, 904360, 883404, 951590]","[1098066, 840361, 883404, 872729, 962568]","[1098066, 840361, 1041259, 872729, 883404]","[999999, 1082185, 1029743, 1127831, 995785]"
9,16,[ 1062973 1082185 13007710],"[1323659, 1127403, 1008237, 1887925, 1004173]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1082185, 981760, 995242, 1029743, 1127831]","[1082185, 981760, 995242, 1029743, 840361]","[981760, 1082185, 995242, 1029743, 9834988]","[999999, 866227, 12263788, 1035843, 1084551]"


In [33]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    
    return precision

In [34]:
precision_at_k(result['random_recommendation'], result['actual'], 5)

0.0

In [35]:
result.apply(lambda row: precision_at_k(result['random_recommendation'], result['actual']), axis=1)

0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
       ... 
2037    0.0
2038    0.0
2039    0.0
2040    0.0
2041    0.0
Length: 2042, dtype: float64

In [36]:
result.apply(lambda x: precision_at_k(result['popular_recommendation'], result['actual']), axis=1).mean()

0.0

In [37]:
result.apply(lambda x: precision_at_k(result['itemitem'], result['actual']), axis=1).mean()

0.0

In [38]:
result.apply(lambda x: precision_at_k(result['cosine'], result['actual']), axis=1).mean()

0.0

In [39]:
result.apply(lambda x: precision_at_k(result['tfidf'], result['actual']), axis=1).mean()

0.0

In [40]:
result.apply(lambda x: precision_at_k(result['own_purchases'], result['actual']), axis=1).mean()

0.0

Прочему то одни нули считает. Может быть нет совпадений и потому precision 0?

In [25]:
data_train.shape[0], data_test.shape[0]

(2278490, 118314)

Попробую еще раз пересчитать random_recommendation и popular_recommendation, а не брать готовые из файла

In [41]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)


,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [42]:
def random_recommendation(items, n=5):
    """Случайные рекоммендации"""
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [43]:
items = data_train.item_id.unique()

result['random_recommendation'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))

In [44]:
def popularity_recommendation(data, n=5):
    """Топ-n популярных товаров"""
    
    popular = data.groupby('item_id')['sales_value'].sum().reset_index()
    popular.sort_values('sales_value', ascending=False, inplace=True)
    
    recs = popular.head(n).item_id
   
    return recs.tolist()

In [45]:
popular_recs = popularity_recommendation(data_train, n=5)

result['popular_recommendation'] = result['user_id'].apply(lambda x: popular_recs)

In [46]:
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[13672325, 15627659, 16767493, 9803285, 871946]","[6534178, 6533889, 1029743, 6534166, 1082185]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[8019721, 5125479, 503776, 15778647, 1060699]","[6534178, 6533889, 1029743, 6534166, 1082185]"


In [47]:
result.apply(lambda x: precision_at_k(result['popular_recommendation'], result['actual']), axis=1).mean()

0.0

In [49]:
result.apply(lambda row: precision_at_k(result['random_recommendation'], result['actual']), axis=1).mean()

0.0

Все равно нули...

### Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.